# FinPlan-LLM: A Multi-Agent Financial Advisory System

**Author:** Harsh Vijay Mamania  
**Course:** DS 5983 - Introduction to Large Language Models  
**Instructor:** Prof. Yehoshua Roi  
**Institution:** Northeastern University, Khoury College of Computer Sciences  
**Date:** December 2025

---

## Abstract

This notebook implements FinPlan-LLM, a multi-agent financial advisory system built on LangGraph. The system demonstrates the core principle: **LLMs for reasoning and soft decisions, heuristics for hard financial decisions**.

The architecture features:
- 8-node workflow with conditional routing
- ReAct-based autonomous research agent
- Real-time macroeconomic data integration via FRED API
- Systematic constraint parsing and ETF filtering
- Historical backtesting with yFinance
- LLM-as-Judge validation with rebalancing loop
- Three-layer explainability (Console, PDF, LangSmith)

---


## Table of Contents

1. [Setup and Dependencies](#1-setup-and-dependencies)
2. [Configuration](#2-configuration)
3. [Data Sources and APIs](#3-data-sources-and-apis)
4. [ETF Universe and Metadata](#4-etf-universe-and-metadata)
5. [Core Components](#5-core-components)
   - 5.1 Profile Elicitor
   - 5.2 Constraint Parser
   - 5.3 Macro Analyzer
   - 5.4 Research Agent
   - 5.5 Portfolio Allocator
   - 5.6 Backtester
   - 5.7 News Analyzer
   - 5.8 Validator and Rebalancer
6. [Workflow Graph](#6-workflow-graph)
7. [PDF Report Generation](#7-pdf-report-generation)
8. [Gradio User Interface](#8-gradio-user-interface)
9. [Evaluation](#9-evaluation)

---


In [1]:
!pip install langchain langchain-google-genai langchain-core langgraph yfinance fredapi pandas numpy reportlab gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.0 MB/s eta 0:00:00


### Basic Setup

In [2]:
import os
from google.colab import files

import json
import re
from datetime import datetime

import numpy as np
import pandas as pd

from fredapi import Fred
import yfinance as yf

import requests
from bs4 import BeautifulSoup

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage
from langchain_core.tracers import LangChainTracer
from langsmith import Client
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

from IPython.display import Image, display
import matplotlib.pyplot as plt
from io import BytesIO

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image, PageBreak, Preformatted
from reportlab.lib.enums import TA_CENTER, TA_LEFT, TA_JUSTIFY

import gradio as gr

# Set your API keys
os.environ["GOOGLE_API_KEY"] = "<your API key here>"
os.environ["FRED_API_KEY"] = "<your API key here>"

# PROFILE ELICITOR Agent


In [3]:

# System prompt that tells the agent its role
profile_prompt = """You are a financial advisor's assistant. Your job is to gather information about the user's financial profile.

Ask the user these questions ONE AT A TIME:
1. What is your age?
2. What is your investment goal? (retirement, house purchase, wealth building, etc.)
3. What is your investment time horizon in years?
4. What is your risk tolerance? (conservative, moderate, aggressive)
5. Do you have any investment constraints or preferences? (e.g., no tobacco stocks, ESG focus)

After gathering all information, summarize it in this JSON format:
{
  "age": <number>,
  "goal": "<string>",
  "time_horizon": <number>,
  "risk_tolerance": "<string>",
  "constraints": "<string>"
}
"""

In [4]:
# Create the profile elicitor (no tools needed, just conversation)
profile_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

### Parse JSON Profile

In [5]:
# Extract JSON from the response (agent might wrap it in markdown)
def extract_json(text):
    # Remove markdown code blocks if present
    text = re.sub(r'```json\s*', '', text)
    text = re.sub(r'```\s*', '', text)
    text = text.strip()
    return json.loads(text)

# MACRO WATCHER Agent

In [6]:
@tool
def get_macro_data(indicator: str) -> str:
    """Get macroeconomic data from FRED.

    Args:
        indicator: The economic indicator to fetch. Options:
                  - 'fed_rate': Federal Funds Rate
                  - 'inflation': Consumer Price Index (CPI)
                  - 'unemployment': Unemployment Rate
                  - 'gdp': GDP Growth Rate
    """
    fred = Fred(api_key=os.environ['FRED_API_KEY'])

    # Map indicator names to FRED series IDs
    series_map = {
        'fed_rate': 'DFF',           # Federal Funds Rate
        'inflation': 'CPIAUCSL',     # CPI for All Urban Consumers
        'unemployment': 'UNRATE',     # Unemployment Rate
        'gdp': 'A191RL1Q225SBEA'     # Real GDP Growth Rate
    }

    try:
        series_id = series_map.get(indicator)
        if not series_id:
            return f"Unknown indicator: {indicator}. Valid options: {list(series_map.keys())}"

        # Get most recent value
        data = fred.get_series(series_id)
        latest_value = data.iloc[-1]
        latest_date = data.index[-1].strftime('%Y-%m-%d')

        return f"{indicator}: {latest_value:.2f} (as of {latest_date})"
    except Exception as e:
        return f"Error fetching {indicator}: {str(e)}"

In [7]:
# Create agent with macro data tool
macro_agent = create_react_agent(
    ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0),
    tools=[get_macro_data]
)

/tmp/ipython-input-855402229.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  macro_agent = create_react_agent(


# DATA RETRIEVER agent

In [8]:
@tool
def get_stock_price (ticker: str) -> str:
  """
  Get the current stock price for a given ticker
  Args:
    ticker: stock ticker symbol (eg: 'AAPL', 'MSFT')
  """
  try:
      stock = yf.Ticker(ticker)
      # Get the most recent price
      hist = stock.history(period="1d")
      if hist.empty:
          return f"Could not find data for ticker {ticker}"

      current_price = hist['Close'].iloc[-1]
      return f"The current price of {ticker} is ${current_price:.2f}"
  except Exception as e:
      return f"Error fetching data for {ticker}: {str(e)}"

In [9]:
@tool
def get_stock_info(ticker: str) -> str:
    """Get detailed stock/ETF information including price, fundamentals, and metrics.

    Args:
        ticker: Stock/ETF ticker symbol (e.g., 'SPY', 'VTI', 'AAPL')
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        hist = stock.history(period="1y")

        if hist.empty:
            return f"No data found for {ticker}"

        # Calculate key metrics
        current_price = hist['Close'].iloc[-1]
        year_start_price = hist['Close'].iloc[0]
        ytd_return = ((current_price - year_start_price) / year_start_price) * 100

        # Extract fundamental data (may not exist for all ETFs)
        pe_ratio = info.get('trailingPE', 'N/A')
        dividend_yield = info.get('dividendYield', 0)
        if dividend_yield and dividend_yield != 'N/A':
            dividend_yield = dividend_yield * 100  # Convert to percentage

        result = f"""
{ticker} Information:
- Current Price: ${current_price:.2f}
- YTD Return: {ytd_return:.2f}%
- P/E Ratio: {pe_ratio}
- Dividend Yield: {dividend_yield:.2f}% if dividend_yield else 'N/A'
- Market Cap: {info.get('marketCap', 'N/A')}
        """

        return result.strip()
    except Exception as e:
        return f"Error fetching {ticker}: {str(e)}"

#### Create nodes

In [10]:
# Step 1: "Profile Collection" Node
def profile_node(state):
  """Collects user profile - using our pre-collected data"""
  print("[STEP] Step 1: Profile Collection")
  state['user_profile'] = state['user_profile'] # user_profile
  return state

# Multiple Asset Classes

### Define Asset Class Universe

In [11]:
# Define candidate ETFs for each asset class
ASSET_UNIVERSE = {
    "US_Stocks": {
        "large_cap": ["SPY", "VOO", "IVV", "VTI"],
        "mid_cap": ["VO", "IJH", "MDY"],
        "small_cap": ["VB", "IWM", "IJR"],
        "growth": ["VUG", "IWF", "VONG"],
        "value": ["VTV", "IWD", "VONV"]
    },
    "International_Stocks": {
        "developed": ["VEA", "IEFA", "VXUS", "EFA"],
        "emerging": ["VWO", "IEMG", "EEM"]
    },
    "Bonds": {
        "total_bond": ["BND", "AGG", "VCIT"],
        "treasury": ["GOVT", "IEF", "TLT"],
        "corporate": ["LQD", "VCIT", "USIG"],
        "tips": ["TIP", "SCHP", "VTIP"]  # Inflation-protected
    },
    "REITs": {
        "diversified": ["VNQ", "IYR", "SCHH", "USRT"],
        "mortgage": ["REM", "RWR"]
    },
    "Commodities": {
        "gold": ["GLD", "IAU", "GLDM"],
        "broad_commodities": ["DBC", "GSG", "PDBC"],
        "energy": ["XLE", "VDE"]
    }
}

In [12]:
# Flatten to get total count
total_etfs = sum(len(etfs) for category in ASSET_UNIVERSE.values()
                 for etfs in category.values())

print(f"Total ETF candidates: {total_etfs}")
print(f"Asset classes: {list(ASSET_UNIVERSE.keys())}")

Total ETF candidates: 49
Asset classes: ['US_Stocks', 'International_Stocks', 'Bonds', 'REITs', 'Commodities']


### ETF METADATA - Tags for constraint filtering

In [13]:
ETF_METADATA = {
    # US Stocks - Large Cap
    "SPY":  {"name": "S&P 500 ETF", "tags": ["us_equity", "large_cap", "broad_market"], "esg": False, "china_exposure": False},
    "VOO":  {"name": "Vanguard S&P 500", "tags": ["us_equity", "large_cap", "broad_market"], "esg": False, "china_exposure": False},
    "IVV":  {"name": "iShares S&P 500", "tags": ["us_equity", "large_cap", "broad_market"], "esg": False, "china_exposure": False},
    "VTI":  {"name": "Vanguard Total Stock", "tags": ["us_equity", "total_market", "broad_market"], "esg": True, "china_exposure": False},

    # US Stocks - Mid/Small Cap
    "VO":   {"name": "Vanguard Mid-Cap", "tags": ["us_equity", "mid_cap"], "esg": False, "china_exposure": False},
    "IJH":  {"name": "iShares Mid-Cap", "tags": ["us_equity", "mid_cap"], "esg": False, "china_exposure": False},
    "MDY":  {"name": "SPDR Mid-Cap", "tags": ["us_equity", "mid_cap"], "esg": False, "china_exposure": False},
    "VB":   {"name": "Vanguard Small-Cap", "tags": ["us_equity", "small_cap"], "esg": False, "china_exposure": False},
    "IWM":  {"name": "iShares Russell 2000", "tags": ["us_equity", "small_cap"], "esg": False, "china_exposure": False},
    "IJR":  {"name": "iShares Small-Cap", "tags": ["us_equity", "small_cap"], "esg": False, "china_exposure": False},

    # US Stocks - Growth/Value
    "VUG":  {"name": "Vanguard Growth", "tags": ["us_equity", "growth", "tech_heavy"], "esg": False, "china_exposure": False},
    "IWF":  {"name": "iShares Growth", "tags": ["us_equity", "growth", "tech_heavy"], "esg": False, "china_exposure": False},
    "VONG": {"name": "Vanguard Russell Growth", "tags": ["us_equity", "growth"], "esg": False, "china_exposure": False},
    "VTV":  {"name": "Vanguard Value", "tags": ["us_equity", "value"], "esg": False, "china_exposure": False},
    "IWD":  {"name": "iShares Value", "tags": ["us_equity", "value"], "esg": False, "china_exposure": False},
    "VONV": {"name": "Vanguard Russell Value", "tags": ["us_equity", "value"], "esg": False, "china_exposure": False},

    # International Stocks
    "VEA":  {"name": "Vanguard Developed Markets", "tags": ["international", "developed"], "esg": False, "china_exposure": False},
    "IEFA": {"name": "iShares Developed Markets", "tags": ["international", "developed"], "esg": False, "china_exposure": False},
    "VXUS": {"name": "Vanguard Total International", "tags": ["international", "broad_market"], "esg": False, "china_exposure": True},
    "EFA":  {"name": "iShares EAFE", "tags": ["international", "developed"], "esg": False, "china_exposure": False},
    "VWO":  {"name": "Vanguard Emerging Markets", "tags": ["international", "emerging", "high_risk"], "esg": False, "china_exposure": True},
    "IEMG": {"name": "iShares Emerging Markets", "tags": ["international", "emerging", "high_risk"], "esg": False, "china_exposure": True},
    "EEM":  {"name": "iShares MSCI Emerging", "tags": ["international", "emerging", "high_risk"], "esg": False, "china_exposure": True},

    # Bonds
    "BND":  {"name": "Vanguard Total Bond", "tags": ["bonds", "fixed_income", "investment_grade"], "esg": True, "china_exposure": False},
    "AGG":  {"name": "iShares Core Bond", "tags": ["bonds", "fixed_income", "investment_grade"], "esg": False, "china_exposure": False},
    "VCIT": {"name": "Vanguard Intermediate Corp", "tags": ["bonds", "corporate", "investment_grade"], "esg": False, "china_exposure": False},
    "GOVT": {"name": "iShares US Treasury", "tags": ["bonds", "treasury", "government", "safe_haven"], "esg": True, "china_exposure": False},
    "IEF":  {"name": "iShares 7-10Y Treasury", "tags": ["bonds", "treasury", "government"], "esg": True, "china_exposure": False},
    "TLT":  {"name": "iShares 20+Y Treasury", "tags": ["bonds", "treasury", "long_duration"], "esg": True, "china_exposure": False},
    "LQD":  {"name": "iShares Investment Grade Corp", "tags": ["bonds", "corporate", "investment_grade"], "esg": False, "china_exposure": False},
    "USIG": {"name": "iShares US IG Corp", "tags": ["bonds", "corporate", "investment_grade"], "esg": False, "china_exposure": False},
    "TIP":  {"name": "iShares TIPS", "tags": ["bonds", "inflation_protected", "treasury"], "esg": True, "china_exposure": False},
    "SCHP": {"name": "Schwab TIPS", "tags": ["bonds", "inflation_protected"], "esg": False, "china_exposure": False},
    "VTIP": {"name": "Vanguard Short TIPS", "tags": ["bonds", "inflation_protected", "short_duration"], "esg": True, "china_exposure": False},

    # REITs
    "VNQ":  {"name": "Vanguard Real Estate", "tags": ["reits", "real_estate", "income"], "esg": False, "china_exposure": False},
    "IYR":  {"name": "iShares Real Estate", "tags": ["reits", "real_estate"], "esg": False, "china_exposure": False},
    "SCHH": {"name": "Schwab REIT", "tags": ["reits", "real_estate"], "esg": False, "china_exposure": False},
    "USRT": {"name": "iShares Core REIT", "tags": ["reits", "real_estate"], "esg": False, "china_exposure": False},
    "REM":  {"name": "iShares Mortgage REIT", "tags": ["reits", "mortgage", "high_yield", "high_risk"], "esg": False, "china_exposure": False},
    "RWR":  {"name": "SPDR Dow Jones REIT", "tags": ["reits", "real_estate"], "esg": False, "china_exposure": False},

    # Commodities
    "GLD":  {"name": "SPDR Gold", "tags": ["commodities", "gold", "precious_metals", "safe_haven"], "esg": True, "china_exposure": False},
    "IAU":  {"name": "iShares Gold", "tags": ["commodities", "gold", "precious_metals"], "esg": True, "china_exposure": False},
    "GLDM": {"name": "SPDR Gold MiniShares", "tags": ["commodities", "gold", "precious_metals"], "esg": True, "china_exposure": False},
    "DBC":  {"name": "Invesco DB Commodity", "tags": ["commodities", "broad_commodities", "oil", "fossil_fuel"], "esg": False, "china_exposure": False},
    "GSG":  {"name": "iShares GSCI Commodity", "tags": ["commodities", "broad_commodities", "oil", "fossil_fuel"], "esg": False, "china_exposure": False},
    "PDBC": {"name": "Invesco Optimum Yield", "tags": ["commodities", "broad_commodities"], "esg": False, "china_exposure": False},
    "XLE":  {"name": "Energy Select SPDR", "tags": ["commodities", "energy", "oil", "fossil_fuel"], "esg": False, "china_exposure": False},
    "VDE":  {"name": "Vanguard Energy", "tags": ["commodities", "energy", "oil", "fossil_fuel"], "esg": False, "china_exposure": False},
}

# Known constraint keywords -> tags mapping
CONSTRAINT_KEYWORDS = {
    # Exclusions
    "no oil": ["oil", "fossil_fuel"],
    "no fossil": ["fossil_fuel", "oil"],
    "no tobacco": ["tobacco"],
    "no weapons": ["weapons", "defense"],
    "no china": ["china_exposure"],
    "no emerging": ["emerging"],
    "no high risk": ["high_risk"],

    # Preferences
    "esg": "esg_required",
    "green": ["clean_energy", "esg"],
    "safe": ["safe_haven", "treasury", "government"],
    "income": ["income", "high_yield"],
    "growth": ["growth"],
    "value": ["value"],
}

### CONSTRAINT PARSER NODE

In [14]:
def constraint_parser_node(state):
    """Parse natural language constraints into structured filters"""
    print("Step: Parsing Investment Constraints")

    constraints_text = state['user_profile'].get('constraints', '')

    if not constraints_text or constraints_text.lower() in ['none', 'no', 'n/a', '']:
        print("  -> No constraints specified")
        state['parsed_constraints'] = {"exclude_tags": [], "require_esg": False, "preferences": []}
        state['excluded_etfs'] = []
        state['filtered_universe'] = ASSET_UNIVERSE.copy()
        return state

    # LLM extracts structured constraints
    parser_prompt = f"""Parse these investment constraints into structured format.

USER CONSTRAINTS: "{constraints_text}"

KNOWN CONSTRAINT TYPES:
- Exclusions: oil/fossil fuels, tobacco, weapons/defense, China exposure, emerging markets, high risk
- Requirements: ESG/sustainable investing, safe/conservative assets
- Preferences: growth, value, income-focused, gold/precious metals

Return ONLY valid JSON (no markdown):
{{
    "exclude_tags": ["tag1", "tag2"],
    "require_esg": true/false,
    "preferences": ["preference1"],
    "unmatched_constraints": ["anything you couldn't parse"]
}}

Examples:
- "No oil, ESG only" -> {{"exclude_tags": ["oil", "fossil_fuel"], "require_esg": true, "preferences": [], "unmatched_constraints": []}}
- "Avoid China, prefer safe assets" -> {{"exclude_tags": ["china_exposure"], "require_esg": false, "preferences": ["safe_haven"], "unmatched_constraints": []}}
"""

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    response = llm.invoke(parser_prompt)

    # Parse LLM response
    try:
        parsed = extract_json(response.content)
    except:
        print(f"  -> Failed to parse LLM response, using fallback")
        parsed = {"exclude_tags": [], "require_esg": False, "preferences": [], "unmatched_constraints": [constraints_text]}

    print(f"  -> Parsed: {parsed}")

    # Filter ETFs based on constraints
    excluded_etfs = []
    for ticker, metadata in ETF_METADATA.items():
        # Check exclusion tags
        for tag in parsed.get('exclude_tags', []):
            if tag in metadata['tags'] or (tag == 'china_exposure' and metadata.get('china_exposure')):
                excluded_etfs.append(ticker)
                break
        if parsed.get('require_esg'):
            state['prefer_esg'] = True  # Soft preference, handled in research agent

    excluded_etfs = list(set(excluded_etfs))
    print(f"  -> Excluded ETFs: {excluded_etfs}")

    # Create filtered universe
    filtered_universe = {}
    for asset_class, subcategories in ASSET_UNIVERSE.items():
        filtered_universe[asset_class] = {}
        for subcategory, tickers in subcategories.items():
            filtered_tickers = [t for t in tickers if t not in excluded_etfs]
            if filtered_tickers:
                filtered_universe[asset_class][subcategory] = filtered_tickers

    # Check for warnings
    warnings = []
    for asset_class, subcategories in filtered_universe.items():
        if not subcategories or all(len(t) == 0 for t in subcategories.values()):
            warnings.append(f"[WARN] No ETFs available for {asset_class} after applying constraints")

    if warnings:
        print(f"  -> Warnings: {warnings}")

    if parsed.get('unmatched_constraints'):
        print(f"  -> Could not fully parse: {parsed['unmatched_constraints']}")

    state['parsed_constraints'] = parsed
    state['excluded_etfs'] = excluded_etfs
    state['filtered_universe'] = filtered_universe
    state['constraint_warnings'] = warnings

    return state

# **Research Agent**

### Step 1: Define Research Tools

In [15]:
@tool
def get_etf_details(ticker: str) -> str:
    """Get detailed information about a specific ETF including price, returns, and fundamentals.

    Args:
        ticker: ETF ticker symbol (e.g., 'SPY', 'BND', 'VTI')
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        hist = stock.history(period="1y")

        if hist.empty:
            return f"No data found for {ticker}"

        current_price = hist['Close'].iloc[-1]
        ytd_return = ((hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]) * 100

        # Calculate volatility
        returns = hist['Close'].pct_change().dropna()
        volatility = returns.std() * np.sqrt(252) * 100

        # Get metadata if available
        metadata = ETF_METADATA.get(ticker, {})

        result = f"""
**{ticker}** - {metadata.get('name', info.get('shortName', 'N/A'))}
- Current Price: ${current_price:.2f}
- YTD Return: {ytd_return:.2f}%
- Volatility: {volatility:.2f}%
- Expense Ratio: {info.get('expenseRatio', 'N/A')}
- AUM: ${info.get('totalAssets', 0)/1e9:.2f}B
- ESG Friendly: {'Yes' if metadata.get('esg') else 'No'}
- Tags: {', '.join(metadata.get('tags', []))}
"""
        return result.strip()
    except Exception as e:
        return f"Error fetching {ticker}: {str(e)}"

In [16]:
@tool
def compare_etfs(ticker1: str, ticker2: str) -> str:
    """Compare two ETFs side by side on key metrics.

    Args:
        ticker1: First ETF ticker symbol
        ticker2: Second ETF ticker symbol
    """
    try:
        results = []

        for ticker in [ticker1, ticker2]:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1y")
            info = stock.info

            if hist.empty:
                results.append({"ticker": ticker, "error": "No data"})
                continue

            returns = hist['Close'].pct_change().dropna()
            ytd_return = ((hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]) * 100
            sharpe = calculate_sharpe_ratio(returns)
            max_dd = ((hist['Close'] / hist['Close'].cummax()) - 1).min() * 100

            results.append({
                "ticker": ticker,
                "ytd_return": ytd_return,
                "sharpe": sharpe,
                "max_drawdown": max_dd,
                "expense_ratio": info.get('expenseRatio', 0) or 0,
                "esg": ETF_METADATA.get(ticker, {}).get('esg', False)
            })

        comparison = f"""
**{ticker1} vs {ticker2} Comparison**

| Metric | {ticker1} | {ticker2} | Winner |
|--------|-----------|-----------|--------|
| YTD Return | {results[0]['ytd_return']:.2f}% | {results[1]['ytd_return']:.2f}% | {ticker1 if results[0]['ytd_return'] > results[1]['ytd_return'] else ticker2} |
| Sharpe Ratio | {results[0]['sharpe']:.2f} | {results[1]['sharpe']:.2f} | {ticker1 if results[0]['sharpe'] > results[1]['sharpe'] else ticker2} |
| Max Drawdown | {results[0]['max_drawdown']:.2f}% | {results[1]['max_drawdown']:.2f}% | {ticker1 if results[0]['max_drawdown'] > results[1]['max_drawdown'] else ticker2} |
| Expense Ratio | {results[0]['expense_ratio']:.4f} | {results[1]['expense_ratio']:.4f} | {ticker1 if results[0]['expense_ratio'] < results[1]['expense_ratio'] else ticker2} |
| ESG | {'Yes' if results[0]['esg'] else 'No'} | {'Yes' if results[1]['esg'] else 'No'} | - |

**Recommendation**: Based on risk-adjusted returns (Sharpe), **{ticker1 if results[0]['sharpe'] > results[1]['sharpe'] else ticker2}** is preferred.
"""
        return comparison.strip()
    except Exception as e:
        return f"Error comparing ETFs: {str(e)}"

In [17]:
@tool
def screen_asset_class(asset_class: str, excluded_etfs: str = "", prefer_esg: bool = False, max_results: int = 3) -> str:
    """Screen and rank ETFs within an asset class based on performance metrics.

    Args:
        asset_class: Asset class to screen (US_Stocks, International_Stocks, Bonds, REITs, Commodities)
        excluded_etfs: Comma-separated ETF tickers to exclude (e.g., "XLE,VDE,DBC")
        prefer_esg: Whether to prioritize ESG-friendly ETFs
        max_results: Number of top ETFs to return
    """
    if asset_class not in ASSET_UNIVERSE:
        return f"Unknown asset class: {asset_class}. Valid: {list(ASSET_UNIVERSE.keys())}"

    excluded_list = [e.strip() for e in excluded_etfs.split(",") if e.strip()]

    # Gather all tickers for this asset class
    all_tickers = []
    for subcategory, tickers in ASSET_UNIVERSE[asset_class].items():
        for t in tickers:
            if t not in excluded_list and t not in all_tickers:
                all_tickers.append(t)

    all_tickers = all_tickers[:10]  # Limit for speed

    # Fetch metrics
    etf_metrics = []
    for ticker in all_tickers:
        metrics = calculate_metrics(ticker, "6mo")
        if metrics:
            metrics['esg'] = ETF_METADATA.get(ticker, {}).get('esg', False)
            etf_metrics.append(metrics)

    if not etf_metrics:
        return f"No valid data for {asset_class}"

    # Score ETFs
    for etf in etf_metrics:
        score = 0
        score += etf['sharpe_ratio'] * 20
        score += etf['ytd_return'] * 0.5
        score += min(etf['aum'] / 1e9, 10)
        score -= abs(etf['max_drawdown']) * 0.3
        score -= etf['expense_ratio'] * 10
        if prefer_esg and etf['esg']:
            score += 10  # ESG bonus
        etf['score'] = score

    # Sort and get top N
    top_etfs = sorted(etf_metrics, key=lambda x: x['score'], reverse=True)[:max_results]

    result = f"\n**Top {max_results} ETFs for {asset_class}**\n"
    for i, etf in enumerate(top_etfs, 1):
        result += f"\n{i}. **{etf['ticker']}** (Score: {etf['score']:.1f})"
        result += f"\n   - YTD Return: {etf['ytd_return']:.2f}%"
        result += f"\n   - Sharpe Ratio: {etf['sharpe_ratio']:.2f}"
        result += f"\n   - Max Drawdown: {etf['max_drawdown']:.2f}%"
        result += f"\n   - ESG: {'[OK]' if etf['esg'] else '[FAIL]'}"

    return result

In [18]:
@tool
def fetch_market_news(query: str) -> str:
    """Fetch recent market news and headlines for a given topic or ETF.

    Args:
        query: Search query (e.g., 'SPY ETF', 'bond market outlook', 'gold prices')
    """
    try:
        # Using Yahoo Finance news as fallback (free, no API key)

        # Search via Yahoo Finance
        url = f"https://finance.yahoo.com/quote/{query}/news"
        headers = {'User-Agent': 'Mozilla/5.0'}

        # Fallback: just return a simulated response for demo
        # In production, integrate NewsAPI or similar
        return f"""
**Recent News for "{query}" -- Fallback**

1. "Market analysts remain cautiously optimistic on {query} amid economic uncertainty"
   - Source: Financial Times

2. "ETF flows show continued interest in {query} category"
   - Source: Bloomberg

3. "Experts weigh in on {query} performance outlook for 2025"
   - Source: Reuters

*Note: For live news, integrate NewsAPI or similar service*
"""
    except Exception as e:
        return f"Error fetching news: {str(e)}"

### RESEARCH AGENT NODE

In [19]:
def research_agent_node(state):
    """Research agent autonomously investigates ETFs for each asset class"""
    print("Step: Research Agent - Autonomous ETF Analysis")

    profile = state['user_profile']
    excluded_etfs = state.get('excluded_etfs', [])
    prefer_esg = state.get('parsed_constraints', {}).get('require_esg', False)
    filtered_universe = state.get('filtered_universe', ASSET_UNIVERSE)

    # Create research agent with tools
    research_tools = [
        screen_asset_class,
        get_etf_details,
        compare_etfs,
        fetch_market_news
    ]

    research_agent = create_react_agent(
        ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0),
        tools=research_tools
    )

    # Build research prompt
    research_prompt = f"""You are a financial research analyst. Research and select the best ETF for each asset class.

**INVESTOR PROFILE:**
- Age: {profile['age']}
- Goal: {profile['goal']}
- Risk Tolerance: {profile['risk_tolerance']}
- Time Horizon: {profile['time_horizon']} years
- ESG Preference: {'Yes - prioritize ESG funds' if prefer_esg else 'No preference'}

**EXCLUDED ETFs (do not recommend):** {excluded_etfs if excluded_etfs else 'None'}

**ASSET CLASSES TO RESEARCH:**
{list(filtered_universe.keys())}

**YOUR TASK:**
1. For each asset class, use screen_asset_class to find top candidates
2. 2. For US_Stocks and Bonds, you MUST call compare_etfs on the top 2 candidates before picking a winner
3. If the investor has specific concerns (e.g., conservative), use get_etf_details for deeper analysis
4. You MUST call fetch_market_news for at least 2 asset classes before finalizing. Include news findings in your summary.

**OUTPUT FORMAT:**
After your research, provide your final picks in this exact format:
FINAL_PICKS:
- US_Stocks: [TICKER]
- International_Stocks: [TICKER]
- Bonds: [TICKER]
- REITs: [TICKER]
- Commodities: [TICKER]

RESEARCH_SUMMARY:
[2-3 sentence summary of your research process and key findings]

Begin your research now.
"""

    print(f"  -> Researching {len(filtered_universe)} asset classes...")
    print(f"  -> Excluded ETFs: {excluded_etfs}")
    print(f"  -> ESG Preference: {prefer_esg}")

    try:
        response = research_agent.invoke({
            "messages": [HumanMessage(content=research_prompt)]},
            config={"recursion_limit": 50}
        )

        research_output = response['messages'][-1].content

        # Handle different response formats from Gemini
        if isinstance(research_output, list):
            research_output = research_output[0].get('text', str(research_output))

        print(f"  -> Research complete!")

        # Parse final picks from response
        top_picks = {}
        if "FINAL_PICKS:" in research_output:
            picks_section = research_output.split("FINAL_PICKS:")[1]
            if "RESEARCH_SUMMARY:" in picks_section:
                picks_section = picks_section.split("RESEARCH_SUMMARY:")[0]

            for line in picks_section.strip().split("\n"):
                line = line.strip().replace("- ", "").replace("*", "")
                if ":" in line:
                    parts = line.split(":")
                    asset_class = parts[0].strip()
                    ticker = parts[1].strip().replace("[", "").replace("]", "").upper()
                    # Validate ticker exists
                    if ticker in ETF_METADATA:
                        top_picks[asset_class] = ticker

        # Fallback: if parsing failed, use defaults from screening
        if len(top_picks) < 3:
            print(f"  -> Partial picks parsed ({len(top_picks)}), filling gaps with screener...")
            for asset_class in filtered_universe.keys():
                if asset_class not in top_picks:
                    # Quick screen to get top pick
                    result = screen_asset_class.invoke({
                        "asset_class": asset_class,
                        "excluded_etfs": ",".join(excluded_etfs),
                        "prefer_esg": prefer_esg,
                        "max_results": 1
                    })
                    # Extract ticker from result
                    for line in result.split("\n"):
                        if "**" in line and "." in line:
                            ticker = line.split("**")[1].strip()
                            if ticker in ETF_METADATA:
                                top_picks[asset_class] = ticker
                                break

        print(f"  -> Final picks: {top_picks}")

        state['research_output'] = research_output
        state['top_picks'] = top_picks
        state['screened_assets'] = {ac: f"Selected: {t}" for ac, t in top_picks.items()}

    except Exception as e:
        print(f"  -> Research agent error: {e}")
        print(f"  -> Falling back to basic screening...")

        # Fallback to basic screening
        top_picks = {}
        screened_assets = {}
        for asset_class in filtered_universe.keys():
            result = screen_asset_class.invoke({
                "asset_class": asset_class,
                "excluded_etfs": ",".join(excluded_etfs),
                "prefer_esg": prefer_esg,
                "max_results": 2
            })
            screened_assets[asset_class] = result
            # Extract top ticker
            for line in result.split("\n"):
                if "1. **" in line:
                    ticker = line.split("**")[1].strip()
                    top_picks[asset_class] = ticker
                    break

        state['research_output'] = "Fallback screening used due to agent error"
        state['top_picks'] = top_picks
        state['screened_assets'] = screened_assets

    return state

### News Analyzer Node

In [20]:
def news_analyzer_node(state):
    """Fetch news for top holdings, summarize, and analyze sentiment"""
    print("Step: News Analysis - Summarization & Sentiment")

    top_picks = state.get('top_picks', {})

    if not top_picks:
        print("  -> No top picks to analyze")
        state['news_analysis'] = "No holdings to analyze"
        state['news_sentiment'] = "NEUTRAL"
        return state

    # Get top 3-4 holdings for news analysis
    holdings = list(top_picks.values())[:4]
    print(f"  -> Analyzing news for: {holdings}")

    # Fetch news for each holding
    all_news = []
    for ticker in holdings:
        news = fetch_market_news.invoke({"query": f"{ticker} ETF"})
        all_news.append(f"**{ticker}:**\n{news}")

    combined_news = "\n\n".join(all_news)

    # LLM: Summarize + Sentiment + Impact in one call
    analysis_prompt = f"""You are a financial news analyst. Analyze the following market news for a portfolio containing: {holdings}

{combined_news}

Provide your analysis in this EXACT format:

**NEWS SUMMARY:**
[3-4 sentence summary of key developments across all holdings]

**SENTIMENT:** [BULLISH / BEARISH / NEUTRAL]

**CONFIDENCE:** [HIGH / MEDIUM / LOW]

**PORTFOLIO IMPACT:**
[2-3 sentences on how this news might affect the portfolio]

**KEY RISKS:**
[1-2 bullet points of risks to watch]

**SOURCES:**
[List the news sources mentioned]
"""

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

    try:
        response = llm.invoke(analysis_prompt)
        analysis = response.content

        # Extract sentiment
        sentiment = "NEUTRAL"
        if "**SENTIMENT:** BULLISH" in analysis or "**SENTIMENT:**BULLISH" in analysis:
            sentiment = "BULLISH"
        elif "**SENTIMENT:** BEARISH" in analysis or "**SENTIMENT:**BEARISH" in analysis:
            sentiment = "BEARISH"

        print(f"  -> Sentiment: {sentiment}")

        state['news_analysis'] = analysis
        state['news_sentiment'] = sentiment

    except Exception as e:
        print(f"  -> Error in news analysis: {e}")
        state['news_analysis'] = f"News analysis failed: {str(e)}"
        state['news_sentiment'] = "NEUTRAL"

    return state

### Conditional Loop + Rebalance

### PRESET PORTFOLIOS FOR REBALANCING

In [21]:
PRESET_PORTFOLIOS = {
    "conservative": {
        "US_Stocks": 20,
        "International_Stocks": 10,
        "Bonds": 50,
        "REITs": 10,
        "Commodities": 10
    },
    "moderate": {
        "US_Stocks": 35,
        "International_Stocks": 15,
        "Bonds": 30,
        "REITs": 10,
        "Commodities": 10
    },
    "aggressive": {
        "US_Stocks": 50,
        "International_Stocks": 25,
        "Bonds": 10,
        "REITs": 10,
        "Commodities": 5
    }
}

### Rebalance Node

In [22]:
def rebalance_node(state):
    """Adjust allocation based on validation concerns - LLM diagnoses, heuristics fix"""
    # Increment counter HERE instead of in router
    state['rebalance_attempts'] = state.get('rebalance_attempts', 0) + 1
    # attempts = state.get('rebalance_attempts', 0)
    attempts = state['rebalance_attempts']
    # print(f" Step: Rebalancing Portfolio (Attempt {attempts})")
    print(f" Step: Rebalancing Portfolio (Attempt {attempts})")

    validation_result = state.get('validation_result', '')

    # LLM diagnoses the issue (soft decision)
    diagnosis_prompt = f"""Based on this portfolio validation feedback, what is the main issue?

VALIDATION FEEDBACK:
{validation_result}

Answer with ONLY one of these:
- TOO_AGGRESSIVE (too much equity/risk for this investor)
- TOO_CONSERVATIVE (too little growth for this investor)
- CONCENTRATION_RISK (too much in one asset class)
- CONSTRAINT_VIOLATION (ESG or other constraints not met)
- ACCEPTABLE (no major issues)

Your answer (one word/phrase only):"""

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

    try:
        response = llm.invoke(diagnosis_prompt)
        diagnosis = response.content.strip().upper()
        print(f"  -> Diagnosis: {diagnosis}")

        # Heuristics apply fix based on diagnosis (hard decision)
        current_risk = state['user_profile'].get('risk_tolerance', 'moderate')

        if "AGGRESSIVE" in diagnosis:
            # Shift to more conservative preset
            if current_risk == "aggressive":
                state['preset_override'] = "moderate"
            else:
                state['preset_override'] = "conservative"
            print(f"  -> Shifting to {state['preset_override']} allocation")

        elif "CONSERVATIVE" in diagnosis:
            # Shift to more aggressive preset
            if current_risk == "conservative":
                state['preset_override'] = "moderate"
            else:
                state['preset_override'] = "aggressive"
            print(f"  -> Shifting to {state['preset_override']} allocation")

        elif "CONCENTRATION" in diagnosis:
            # Use moderate as balanced default
            state['preset_override'] = "moderate"
            print(f"  -> Shifting to balanced allocation")

        elif "CONSTRAINT" in diagnosis:
            # Flag for manual review, keep current
            state['constraint_warning'] = "Portfolio may not fully meet constraints. Manual review recommended."
            print(f"  -> Constraint issue flagged for review")

        else:
            # ACCEPTABLE or unknown - no change needed
            print(f"  -> No rebalancing needed")
            state['preset_override'] = None

    except Exception as e:
        print(f"  -> Diagnosis error: {e}")
        state['preset_override'] = "moderate"  # Safe fallback

    return state

### Conditional Routing Function


In [23]:
def should_continue(state):
    """Decide whether to rebalance or continue to report"""
    attempts = state.get('rebalance_attempts', 0)
    validation_status = state.get('validation_status', '')

    # Max 2 rebalance attempts
    if attempts >= 2:
        print(f"  -> Max rebalance attempts ({attempts}) reached. Continuing with disclaimer.")
        state['rebalance_warning'] = "Portfolio was rebalanced multiple times but validator still has concerns. Human review recommended."
        return "continue"

    # Check if validation raised concerns
    if "[WARN]" in validation_status or "CONCERNS" in validation_status.upper():
        # state['rebalance_attempts'] = attempts + 1
        print(f"  -> Validation concerns detected. Triggering rebalance (attempt {attempts + 1})")
        return "rebalance"

    print(f"  -> Validation passed. Continuing to report.")
    return "continue"

In [24]:
@tool
def calculate_multi_asset_allocation(age: int, risk_tolerance: str, goal: str, time_horizon: int) -> dict:
    """Calculate portfolio allocation across multiple asset classes.

    Args:
        age: User's age in years
        risk_tolerance: 'conservative', 'moderate', or 'aggressive'
        goal: Investment goal (e.g., 'retirement', 'house', 'wealth building')
        time_horizon: Investment time horizon in years
    """

    # Base allocation from age
    base_stock_pct = 100 - age

    # Adjust for risk tolerance
    if risk_tolerance.lower() == 'conservative':
        stock_pct = max(20, base_stock_pct - 20)
    elif risk_tolerance.lower() == 'aggressive':
        stock_pct = min(90, base_stock_pct + 20)
    else:  # moderate
        stock_pct = base_stock_pct

    # Adjust for time horizon (if short-term goal, reduce stocks)
    if time_horizon < 5:
        stock_pct = max(20, stock_pct - 20)
    elif time_horizon > 30:
        stock_pct = min(90, stock_pct + 10)

    # Goal-specific adjustments
    if goal.lower() in ['house', 'house purchase', 'down payment']:
        # Short-term goal: more conservative
        stock_pct = max(20, stock_pct - 15)
        bond_pct = 100 - stock_pct
        allocation = {
            "US_Stocks": stock_pct * 0.5,
            "International_Stocks": stock_pct * 0.2,
            "Bonds": bond_pct * 0.8,
            "REITs": stock_pct * 0.3,
            "Commodities": bond_pct * 0.2
        }
    elif goal.lower() in ['retirement', 'long-term']:
        # Long-term: diversified
        bond_pct = 100 - stock_pct
        allocation = {
            "US_Stocks": stock_pct * 0.6,
            "International_Stocks": stock_pct * 0.3,
            "Bonds": bond_pct,
            "REITs": stock_pct * 0.1,
            "Commodities": min(10, bond_pct * 0.15)
        }
    else:  # wealth building, general
        bond_pct = 100 - stock_pct
        allocation = {
            "US_Stocks": stock_pct * 0.55,
            "International_Stocks": stock_pct * 0.25,
            "Bonds": bond_pct * 0.85,
            "REITs": stock_pct * 0.15,
            "Commodities": bond_pct * 0.15 + stock_pct * 0.05
        }

    # Normalize to ensure sum = 100%
    total = sum(allocation.values())
    allocation = {k: (v/total)*100 for k, v in allocation.items()}

    return allocation

### Asset Screener Agent

- Performance Calculation Helper

In [25]:

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    """Calculate Sharpe ratio from returns"""
    excess_returns = returns - risk_free_rate/252  # Daily risk-free rate
    if excess_returns.std() == 0:
        return 0
    return np.sqrt(252) * excess_returns.mean() / excess_returns.std()

def calculate_metrics(ticker, period="1y"):
    """Calculate performance metrics for an ETF"""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period=period)
        info = stock.info

        if hist.empty or len(hist) < 20:
            return None

        # Calculate returns
        returns = hist['Close'].pct_change().dropna()

        # Metrics
        ytd_return = ((hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]) * 100
        sharpe = calculate_sharpe_ratio(returns)
        max_drawdown = ((hist['Close'] / hist['Close'].cummax()) - 1).min() * 100

        metrics = {
            'ticker': ticker,
            'ytd_return': ytd_return,
            'sharpe_ratio': sharpe,
            'max_drawdown': max_drawdown,
            'expense_ratio': info.get('expenseRatio', 0) * 100 if info.get('expenseRatio') else 0.5,
            'aum': info.get('totalAssets', 0),
            'avg_volume': hist['Volume'].mean()
        }

        return metrics
    except:
        return None

- Asset Screener Tool

In [26]:
@tool
def screen_etfs(asset_class: str, max_results: int = 3) -> str:
    """Screen and rank ETFs within an asset class based on performance and quality.

    Args:
        asset_class: Asset class to screen (US_Stocks, International_Stocks, Bonds, REITs, Commodities)
        max_results: Number of top ETFs to return (default 3)
    """

    if asset_class not in ASSET_UNIVERSE:
        return f"Unknown asset class: {asset_class}"

    # Get limited tickers (top ETFs from each subcategory)
    all_tickers = []
    for subcategory, tickers in ASSET_UNIVERSE[asset_class].items():
        all_tickers.append(tickers[0])  # Take only first from each subcategory

    all_tickers = all_tickers[:8]  # Max 8 ETFs

    print(f"Screening {len(all_tickers)} ETFs: {all_tickers}")

    # Fetch metrics sequentially (more reliable)
    etf_metrics = []
    for ticker in all_tickers:
        print(f"Fetching {ticker}...", end=" ")
        metrics = calculate_metrics(ticker, "6mo")
        if metrics:
            etf_metrics.append(metrics)
            print("[OK]")
        else:
            print("[FAIL]")

    if not etf_metrics:
        return f"No valid data for {asset_class}"

    # Score ETFs
    for etf in etf_metrics:
        score = 0
        score += etf['sharpe_ratio'] * 20
        score += etf['ytd_return'] * 0.5
        score += min(etf['aum'] / 1e9, 10)
        score -= abs(etf['max_drawdown']) * 0.3
        score -= etf['expense_ratio'] * 10
        etf['score'] = score

    # Sort and get top N
    top_etfs = sorted(etf_metrics, key=lambda x: x['score'], reverse=True)[:max_results]

    # Format output
    result = f"\n=== Top {max_results} ETFs for {asset_class} ===\n"
    for i, etf in enumerate(top_etfs, 1):
        result += f"\n{i}. {etf['ticker']}"
        result += f"\n   YTD Return: {etf['ytd_return']:.2f}%"
        result += f"\n   Sharpe Ratio: {etf['sharpe_ratio']:.2f}"
        result += f"\n   Max Drawdown: {etf['max_drawdown']:.2f}%"
        result += f"\n   Expense Ratio: {etf['expense_ratio']:.2f}%"
        result += f"\n   Score: {etf['score']:.2f}"

    return result

- Screener Node for Workflow

In [27]:
def screener_node(state):
    """Screen ETFs for each asset class"""
    print("Step: Asset Screening")

    screened_assets = {}

    # Screen each asset class
    for asset_class in ASSET_UNIVERSE.keys():
        print(f"\nScreening {asset_class}...")
        result = screen_etfs.invoke({"asset_class": asset_class, "max_results": 2})
        screened_assets[asset_class] = result

    state['screened_assets'] = screened_assets
    return state

# Enhanced Allocator Agent (Uses Profile + Macro + Screened Results)

- Calculate allocation using profile + macro + screened assets

In [28]:
def enhanced_allocation_node(state):
    """Calculate allocation using profile + macro + screened assets OR preset override"""
    print("Step: Enhanced Portfolio Allocation")

    profile = state['user_profile']
    macro = state['macro_data']
    screened = state['screened_assets']

    # Check if this is a reflection loop with preset override
    preset_override = state.get('preset_override', None)

    if preset_override and preset_override in PRESET_PORTFOLIOS:
        print(f"  -> Using preset: {preset_override} (from reflection loop)")
        final_alloc = PRESET_PORTFOLIOS[preset_override].copy()
        regime = state.get('macro_regime', 'Unknown')

    else:
        # Original allocation logic
        base_alloc = calculate_multi_asset_allocation.invoke({
            "age": profile['age'],
            "risk_tolerance": profile['risk_tolerance'],
            "goal": profile['goal'],
            "time_horizon": profile['time_horizon']
        })

        # Parse macro factors
        fed_rate = float(re.search(r'[\d.]+', macro['fed_rate']).group())

        # Fetch additional macro data
        unemployment_data = get_macro_data.invoke({"indicator": "unemployment"})
        gdp_data = get_macro_data.invoke({"indicator": "gdp"})

        unemployment = float(re.search(r'[\d.]+', unemployment_data).group())
        gdp_growth = float(re.search(r'[\d.]+', gdp_data).group())

        inflation = 3.0  # Simplified

        print(f"Macro: Fed={fed_rate:.2f}%, Inflation~{inflation:.1f}%, Unemployment={unemployment:.1f}%, GDP={gdp_growth:.1f}%")

        # Economic Regime Detection
        adjustments = {k: 0 for k in base_alloc.keys()}
        regime = "Balanced"

        # Regime logic (keeping existing)
        # 1. STAGFLATION (worst case)
        if gdp_growth < 1.0 and inflation > 4.0 and unemployment > 5.0:
            regime = "Stagflation"
            adjustments['Commodities'] = 15
            adjustments['Bonds'] = 10
            adjustments['US_Stocks'] = -15
            adjustments['International_Stocks'] = -10
            adjustments['REITs'] = -10
        # 2. RECESSION
        elif gdp_growth < 0 and unemployment > 6.0:
            regime = "Recession"
            adjustments['Bonds'] = 15
            adjustments['Commodities'] = 5
            adjustments['US_Stocks'] = -15
            adjustments['REITs'] = -5
        # 3. STRONG GROWTH
        elif gdp_growth > 2.5 and unemployment < 5.0 and inflation < 3.0:
            regime = "Strong Growth"
            adjustments['US_Stocks'] = 10
            adjustments['REITs'] = 5
            adjustments['Bonds'] = -10
            adjustments['Commodities'] = -5
        # 4. HIGH INFLATION (standalone)
        elif inflation > 4.0:
            regime = "High Inflation"
            adjustments['Commodities'] = 10
            adjustments['Bonds'] = 10
            adjustments['US_Stocks'] = -10
            adjustments['International_Stocks'] = -5
        # 5. RISING RATES
        elif fed_rate > 4.0:
            regime = "Rising Rates"
            adjustments['Bonds'] = 5
            adjustments['US_Stocks'] = -3
            adjustments['Commodities'] = -2
        else:
            # 6. GOLDILOCKS (default)
            regime = "Goldilocks"

        # Apply adjustments
        final_alloc = base_alloc.copy()
        for asset_class, adjustment in adjustments.items():
            if asset_class in final_alloc:
                final_alloc[asset_class] += adjustment

        # Ensure no negatives
        final_alloc = {k: max(0, v) for k, v in final_alloc.items()}

        # Normalize to 100%
        total = sum(final_alloc.values())
        if total > 0:
            final_alloc = {k: (v/total)*100 for k, v in final_alloc.items()}

        state['macro_regime'] = regime

    # Extract top picks from screened results (same for both paths)
    top_picks = state.get('top_picks', {})

    if not top_picks:
        # Fallback: extract from screened results (old format)
        for asset_class, screen_result in screened.items():
            lines = screen_result.split('\n')
            for line in lines:
                if line.strip().startswith('1.'):
                    ticker = line.split()[1]
                    top_picks[asset_class] = ticker
                    break
        lines = screen_result.split('\n')
        for line in lines:
            if line.strip().startswith('1.'):
                ticker = line.split()[1]
                top_picks[asset_class] = ticker
                break

    # Build allocation text
    allocation_text = f"## Portfolio Allocation\n\n"
    if preset_override:
        allocation_text += f"**Strategy**: {preset_override.title()} (Reflection Loop Override)\n\n"
    else:
        allocation_text += f"**Economic Regime**: {regime}\n\n"

    for asset_class, percentage in sorted(final_alloc.items(), key=lambda x: -x[1]):
        if percentage > 0.5:
            etf = top_picks.get(asset_class, "N/A")
            allocation_text += f"- **{asset_class}**: {percentage:.1f}% -> {etf}\n"

    if not preset_override:
        allocation_text += f"\n**Macro Context**:\n"
        allocation_text += f"- Fed Rate: {fed_rate:.2f}%\n"
        allocation_text += f"- Inflation: ~{inflation:.1f}%\n"
        allocation_text += f"- Unemployment: {unemployment:.1f}%\n"
        allocation_text += f"- GDP Growth: {gdp_growth:.1f}%\n"

    state['allocation'] = allocation_text
    state['final_allocation'] = final_alloc
    state['top_picks'] = top_picks

    return state

# Add Reasoning Traces Node

In [29]:
def reasoning_traces_node(state):
    """Generate comprehensive reasoning traces for the portfolio decision"""
    print("Step: Generate Reasoning Traces")

    profile = state['user_profile']
    final_alloc = state.get('final_allocation', {})
    top_picks = state.get('top_picks', {})
    screened = state.get('screened_assets', {})
    macro_regime = state.get('macro_regime', 'Unknown')
    macro_interpretation = state.get('macro_interpretation', '')
    news_analysis = state.get('news_analysis', '')
    news_sentiment = state.get('news_sentiment', 'NEUTRAL')
    parsed_constraints = state.get('parsed_constraints', {})
    excluded_etfs = state.get('excluded_etfs', [])
    research_output = state.get('research_output', '')
    rebalance_attempts = state.get('rebalance_attempts', 0)

    print(f"  -> DEBUG: Age={profile['age']}, Goal={profile['goal']}")

    reasoning = "## Decision Reasoning\n\n"

    # 1. User Profile Analysis
    reasoning += "### 1. User Profile Analysis\n"
    reasoning += f"- **Age {profile['age']}**: Base stock allocation = {100 - profile['age']}% (100 - age rule)\n"

    risk = profile.get('risk_tolerance', 'moderate')
    if risk == 'conservative':
        reasoning += f"- **Risk Tolerance ({risk})**: Reduced equity exposure by 20%\n"
    elif risk == 'aggressive':
        reasoning += f"- **Risk Tolerance ({risk})**: Increased equity exposure by 20%\n"
    else:
        reasoning += f"- **Risk Tolerance ({risk})**: No adjustment (moderate baseline)\n"

    goal = profile.get('goal', 'general')
    if 'retirement' in goal.lower():
        reasoning += f"- **Goal ({goal})**: Long-term diversified strategy across all asset classes\n"
    elif 'house' in goal.lower():
        reasoning += f"- **Goal ({goal})**: Conservative approach, reduced stock exposure for capital preservation\n"
    else:
        reasoning += f"- **Goal ({goal})**: Balanced growth strategy\n"

    horizon = profile.get('time_horizon', 10)
    if horizon < 5:
        reasoning += f"- **Time Horizon ({horizon} years)**: Short-term -> conservative approach\n"
    elif horizon > 20:
        reasoning += f"- **Time Horizon ({horizon} years)**: Long-term -> growth-oriented approach\n"
    else:
        reasoning += f"- **Time Horizon ({horizon} years)**: Medium-term -> balanced approach\n"

    # 2. Constraints Applied
    if parsed_constraints or excluded_etfs:
        reasoning += "\n### 2. Constraints Applied\n"
        if parsed_constraints.get('exclude_tags'):
            reasoning += f"- **Exclusions**: {', '.join(parsed_constraints['exclude_tags'])}\n"
        if parsed_constraints.get('require_esg'):
            reasoning += f"- **ESG Preference**: Yes (prioritized in selection)\n"
        if excluded_etfs:
            reasoning += f"- **Excluded ETFs**: {', '.join(excluded_etfs[:10])}"
            if len(excluded_etfs) > 10:
                reasoning += f" (+{len(excluded_etfs)-10} more)"
            reasoning += "\n"

    # 3. Economic Environment
    reasoning += f"\n### 3. Economic Environment ({macro_regime})\n"
    if macro_interpretation:
        reasoning += f"{macro_interpretation}\n\n"
    else:
        reasoning += f"- **Fed Rate**: Influences bond yields and borrowing costs\n"
        reasoning += f"- **GDP Growth**: Indicates economic expansion/contraction\n"
        reasoning += f"- **Unemployment**: Reflects labor market health\n"
    reasoning += f"\n**Regime Conclusion**: {macro_regime} -> "

    if macro_regime == "Goldilocks":
        reasoning += "Optimal conditions for balanced portfolio\n"
    elif macro_regime == "Stagflation":
        reasoning += "Defensive positioning, increased commodities\n"
    elif macro_regime == "Recession":
        reasoning += "Flight to safety, increased bonds\n"
    elif macro_regime == "Strong Growth":
        reasoning += "Pro-growth positioning, increased equities\n"
    else:
        reasoning += "Standard allocation approach\n"

    # 4. Research & ETF Selection
    reasoning += "\n### 4. Research & ETF Selection\n"
    if research_output and "RESEARCH_SUMMARY:" in research_output:
        summary = research_output.split("RESEARCH_SUMMARY:")[1].strip()
        reasoning += f"{summary}\n\n"

    reasoning += "**Final ETF Picks:**\n"
    for asset_class, percentage in sorted(final_alloc.items(), key=lambda x: -x[1]):
        if percentage > 0.5:
            etf = top_picks.get(asset_class, "N/A")
            reasoning += f"- **{asset_class}** ({percentage:.1f}%) -> **{etf}**\n"

    # 5. News & Sentiment
    if news_analysis:
        reasoning += f"\n### 5. Market News & Sentiment\n"
        reasoning += f"**Overall Sentiment**: {news_sentiment}\n\n"
        # Extract just the summary from news analysis
        if "**NEWS SUMMARY:**" in news_analysis:
            summary_start = news_analysis.find("**NEWS SUMMARY:**") + len("**NEWS SUMMARY:**")
            summary_end = news_analysis.find("**SENTIMENT:**")
            if summary_end > summary_start:
                news_summary = news_analysis[summary_start:summary_end].strip()
                reasoning += f"{news_summary}\n"

    # 6. Rebalancing (if occurred)
    if rebalance_attempts > 0:
        reasoning += f"\n### 6. Portfolio Rebalancing\n"
        reasoning += f"- **Rebalance Attempts**: {rebalance_attempts}\n"
        reasoning += f"- **Reason**: Validation concerns required allocation adjustment\n"
        if state.get('preset_override'):
            reasoning += f"- **Applied Strategy**: {state['preset_override'].title()} preset\n"

    state['reasoning'] = reasoning
    return state

# Backtesting Engine

In [30]:
def backtest_portfolio(tickers_with_weights, period="1y"):
    """Backtest a portfolio over historical period

    Args:
        tickers_with_weights: dict like {'SPY': 45.1, 'VWO': 20.5, ...}
        period: Historical period (1y, 2y, 3y, max)
    """

    print(f"Backtesting portfolio over {period}...")

    # Fetch historical data for all tickers
    portfolio_data = {}
    for ticker in tickers_with_weights.keys():
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period=period)
            if not hist.empty:
                portfolio_data[ticker] = hist['Close']
        except:
            print(f"Failed to fetch {ticker}")

    if not portfolio_data:
        return None

    # Combine into DataFrame
    df = pd.DataFrame(portfolio_data)
    df = df.dropna()  # Remove days with missing data

    if len(df) < 20:
        return None

    # Calculate daily returns
    returns = df.pct_change().dropna()

    # Calculate weighted portfolio returns
    weights = np.array([tickers_with_weights[ticker]/100 for ticker in df.columns])
    portfolio_returns = (returns * weights).sum(axis=1)

    # Calculate metrics
    total_return = ((1 + portfolio_returns).prod() - 1) * 100
    annual_return = ((1 + portfolio_returns.mean()) ** 252 - 1) * 100
    volatility = portfolio_returns.std() * np.sqrt(252) * 100
    sharpe = calculate_sharpe_ratio(portfolio_returns)

    # Max drawdown
    cumulative = (1 + portfolio_returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min() * 100

    # Best/worst days
    best_day = portfolio_returns.max() * 100
    worst_day = portfolio_returns.min() * 100

    metrics = {
        'period': period,
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown,
        'best_day': best_day,
        'worst_day': worst_day,
        'num_days': len(portfolio_returns)
    }

    return metrics

In [31]:
def backtest_node(state):
    """Backtest the portfolio allocation"""
    print("Step: Backtesting Portfolio")

    final_alloc = state['final_allocation']
    top_picks = state['top_picks']

    # Build ticker->weight mapping
    portfolio = {}
    for asset_class, percentage in final_alloc.items():
        if percentage > 0.5:  # Only backtest meaningful allocations
            ticker = top_picks.get(asset_class)
            if ticker:
                portfolio[ticker] = percentage

    # Run backtest
    backtest_1y = backtest_portfolio(portfolio, "1y")
    backtest_3y = backtest_portfolio(portfolio, "3y")

    # Format results
    backtest_text = "## Backtesting Results\n\n"

    if backtest_1y:
        backtest_text += "### 1-Year Performance\n"
        backtest_text += f"- **Total Return**: {backtest_1y['total_return']:.2f}%\n"
        backtest_text += f"- **Annualized Return**: {backtest_1y['annual_return']:.2f}%\n"
        backtest_text += f"- **Volatility**: {backtest_1y['volatility']:.2f}%\n"
        backtest_text += f"- **Sharpe Ratio**: {backtest_1y['sharpe_ratio']:.2f}\n"
        backtest_text += f"- **Max Drawdown**: {backtest_1y['max_drawdown']:.2f}%\n"
        backtest_text += f"- **Best Day**: +{backtest_1y['best_day']:.2f}%\n"
        backtest_text += f"- **Worst Day**: {backtest_1y['worst_day']:.2f}%\n\n"

    if backtest_3y:
        backtest_text += "### 3-Year Performance\n"
        backtest_text += f"- **Total Return**: {backtest_3y['total_return']:.2f}%\n"
        backtest_text += f"- **Annualized Return**: {backtest_3y['annual_return']:.2f}%\n"
        backtest_text += f"- **Sharpe Ratio**: {backtest_3y['sharpe_ratio']:.2f}\n"
        backtest_text += f"- **Max Drawdown**: {backtest_3y['max_drawdown']:.2f}%\n\n"

    # Benchmark comparison (SPY)
    spy_metrics = backtest_portfolio({'SPY': 100}, "1y")
    if spy_metrics and backtest_1y:
        backtest_text += "### vs. S&P 500 Benchmark (1Y)\n"
        backtest_text += f"- **Portfolio**: {backtest_1y['total_return']:.2f}% (Sharpe {backtest_1y['sharpe_ratio']:.2f})\n"
        backtest_text += f"- **SPY**: {spy_metrics['total_return']:.2f}% (Sharpe {spy_metrics['sharpe_ratio']:.2f})\n"
        outperformance = backtest_1y['total_return'] - spy_metrics['total_return']
        backtest_text += f"- **Outperformance**: {outperformance:+.2f}%\n"

    state['backtest'] = backtest_text
    state['backtest_metrics'] = {'1y': backtest_1y, '3y': backtest_3y}

    return state

# Validation Node

In [32]:
def validation_node(state):
    """Quality assurance - validates portfolio meets standards"""
    print("[OK] Step: Portfolio Validation")

    profile = state['user_profile']
    final_alloc = state['final_allocation']
    top_picks = state['top_picks']
    backtest_metrics = state.get('backtest_metrics', {})

    # Build context for LLM
    context = f"""
USER PROFILE:
- Age: {profile['age']} years
- Investment Goal: {profile['goal']}
- Time Horizon: {profile['time_horizon']} years
- Risk Tolerance: {profile['risk_tolerance']}
- Constraints: {profile.get('constraints', 'None')}

PROPOSED PORTFOLIO:
"""

    for asset_class, pct in sorted(final_alloc.items(), key=lambda x: -x[1]):
        if pct > 0.5:
            etf = top_picks.get(asset_class, 'N/A')
            context += f"- {asset_class}: {pct:.1f}% (ETF: {etf})\n"

    # if backtest_metrics and '1y' in backtest_metrics:
    #     bt = backtest_metrics['1y']
    #     context += f"\nHISTORICAL PERFORMANCE (1 Year):\n"
    if backtest_metrics and backtest_metrics.get('1y'):
        bt = backtest_metrics['1y']
        context += f"\nHISTORICAL PERFORMANCE (1 Year):\n"
        context += f"- Sharpe Ratio: {bt['sharpe_ratio']:.2f}\n"
        context += f"- Max Drawdown: {bt['max_drawdown']:.2f}%\n"
        context += f"- Volatility: {bt['volatility']:.2f}%\n"

    # LLM validation prompt with Chain-of-Thought
    validation_prompt = f"""{context}

TASK: Validate this portfolio using step-by-step analysis.

STEP 1 - Age Appropriateness:
Does the equity allocation make sense for a {profile['age']}-year-old investor?
Consider: Typical rule is 100-age = stocks%, are we within reasonable range?

STEP 2 - Goal Alignment:
Is this portfolio suitable for "{profile['goal']}" over {profile['time_horizon']} years?
Consider: Short-term goals need stability, long-term can handle volatility.

STEP 3 - Risk Tolerance Match:
Does this align with "{profile['risk_tolerance']}" risk preference?
Consider: Stock/bond ratio, volatility levels, drawdown tolerance.

STEP 4 - Constraint Compliance:
Does it respect: "{profile.get('constraints', 'None')}"?
Consider: ESG requirements, sector exclusions, ethical investing.

STEP 5 - Risk Metrics:
Are the backtest metrics acceptable?
Consider: Sharpe >0.5 is good, drawdown <-20% concerning, volatility reasonable.

FINAL VERDICT:
Provide overall assessment: [OK] APPROVED or [WARN] CONCERNS NOTED

FORMAT YOUR RESPONSE AS:
STATUS: [[OK] APPROVED / [WARN] CONCERNS]
SUMMARY: [2-3 sentence overall assessment]
RECOMMENDATIONS: [Any suggestions for improvement, or "None" if approved]
"""

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

    try:
        response = llm.invoke(validation_prompt)
        validation_result = response.content

        # Parse status
        if "[OK]" in validation_result or "APPROVED" in validation_result.upper():
            status = "[OK] APPROVED"
        else:
            status = "[WARN] CONCERNS NOTED"

        print(f"  -> {status}")

        state['validation_result'] = validation_result
        state['validation_status'] = status

    except Exception as e:
        print(f"  -> Error: {e}")
        state['validation_result'] = f"[WARN] Validation failed due to error: {str(e)}"
        state['validation_status'] = "[WARN] ERROR"

    return state

### UPDATED MACRO NODE WITH LLM INTERPRETATION

In [33]:
def macro_node(state):
    """Analyzes macroeconomic conditions with LLM interpretation"""
    print("[STEP] Step 2: Macro Analysis")

    # Hardcoded fetch (keeps heuristics working)
    fed_rate = get_macro_data.invoke({"indicator": "fed_rate"})
    inflation = get_macro_data.invoke({"indicator": "inflation"})
    unemployment = get_macro_data.invoke({"indicator": "unemployment"})
    gdp = get_macro_data.invoke({"indicator": "gdp"})

    state['macro_data'] = {
        "fed_rate": fed_rate,
        "inflation": inflation,
        "unemployment": unemployment,
        "gdp": gdp
    }

    # NEW: LLM interprets the macro environment
    interpretation_prompt = f"""You are a macro economist. Analyze these economic indicators:

- {fed_rate}
- {inflation}
- {unemployment}
- {gdp}

Provide a 2-3 sentence interpretation of the current economic environment
and what it means for investors. Be concise and actionable."""

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

    try:
        response = llm.invoke(interpretation_prompt)
        state['macro_interpretation'] = response.content
        print(f"  -> LLM Interpretation: {response.content[:100]}...")
    except Exception as e:
        state['macro_interpretation'] = "Economic analysis unavailable."
        print(f"  -> Interpretation error: {e}")

    return state

### Report Node

In [34]:
def report_node(state):
    """Generate comprehensive final report with all new features"""
    print("Step: Final Report Generation")

    profile = state['user_profile']
    final_alloc = state.get('final_allocation', {})
    top_picks = state.get('top_picks', {})
    macro_regime = state.get('macro_regime', 'Unknown')
    macro_interpretation = state.get('macro_interpretation', '')
    news_analysis = state.get('news_analysis', '')
    news_sentiment = state.get('news_sentiment', 'NEUTRAL')
    backtest = state.get('backtest', '')
    validation_result = state.get('validation_result', '')
    validation_status = state.get('validation_status', '')
    reasoning = state.get('reasoning', '')
    rebalance_attempts = state.get('rebalance_attempts', 0)
    rebalance_warning = state.get('rebalance_warning', '')
    parsed_constraints = state.get('parsed_constraints', {})
    excluded_etfs = state.get('excluded_etfs', [])

    # Build comprehensive report
    report = """

              FINANCIAL PLANNING REPORT


"""

    # Section 1: Executive Summary
    report += "## [STEP] Executive Summary\n\n"
    report += f"**Client Profile**: {profile['age']} years old, {profile['risk_tolerance']} risk tolerance\n"
    report += f"**Investment Goal**: {profile['goal']}\n"
    report += f"**Time Horizon**: {profile['time_horizon']} years\n"
    report += f"**Constraints**: {profile.get('constraints', 'None')}\n\n"

    # Portfolio at a glance
    report += "**Portfolio at a Glance:**\n"
    for asset_class, percentage in sorted(final_alloc.items(), key=lambda x: -x[1]):
        if percentage > 0.5:
            etf = top_picks.get(asset_class, "N/A")
            report += f"- {asset_class}: {percentage:.1f}% -> {etf}\n"
    report += "\n"

    report += f"**Market Sentiment**: {news_sentiment}\n"
    report += f"**Economic Regime**: {macro_regime}\n"
    report += f"**Validation Status**: {validation_status}\n"

    report += "\n---\n\n"

    # Section 2: Economic Analysis
    report += "## [STEP] Economic Analysis\n\n"
    report += f"**Current Regime**: {macro_regime}\n\n"
    if macro_interpretation:
        report += f"**LLM Interpretation**:\n{macro_interpretation}\n\n"

    macro_data = state.get('macro_data', {})
    if macro_data:
        report += "**Key Indicators**:\n"
        for indicator, value in macro_data.items():
            report += f"- {value}\n"

    report += "\n---\n\n"

    # Section 3: Constraints & Filtering
    if parsed_constraints or excluded_etfs:
        report += "## [STEP] Constraint Analysis\n\n"
        report += "**Parsed Constraints**:\n"
        if parsed_constraints.get('exclude_tags'):
            report += f"- Exclusions: {', '.join(parsed_constraints['exclude_tags'])}\n"
        if parsed_constraints.get('require_esg'):
            report += f"- ESG Preference: Required\n"
        if parsed_constraints.get('preferences'):
            report += f"- Preferences: {', '.join(parsed_constraints['preferences'])}\n"
        report += f"\n**Excluded ETFs** ({len(excluded_etfs)} total): {', '.join(excluded_etfs[:8])}"
        if len(excluded_etfs) > 8:
            report += f" ..."
        report += "\n\n---\n\n"

    # Section 4: Portfolio Allocation
    report += "## [STEP] Portfolio Allocation\n\n"
    report += f"**Strategy**: "
    if state.get('preset_override'):
        report += f"{state['preset_override'].title()} (Rebalanced)\n\n"
    else:
        report += f"Calculated based on profile + macro conditions\n\n"

    report += "| Asset Class | Allocation | ETF | Description |\n"
    report += "|-------------|------------|-----|-------------|\n"
    for asset_class, percentage in sorted(final_alloc.items(), key=lambda x: -x[1]):
        if percentage > 0.5:
            etf = top_picks.get(asset_class, "N/A")
            etf_name = ETF_METADATA.get(etf, {}).get('name', 'N/A')
            report += f"| {asset_class} | {percentage:.1f}% | {etf} | {etf_name} |\n"

    report += "\n---\n\n"

    # Section 5: Research Summary
    research_output = state.get('research_output', '')
    if research_output and "RESEARCH_SUMMARY:" in research_output:
        report += "## [STEP] Research Summary\n\n"
        summary = research_output.split("RESEARCH_SUMMARY:")[1].strip()
        report += f"{summary}\n\n"
        report += "---\n\n"

    # Section 6: News & Sentiment
    if news_analysis:
        report += "## [STEP] Market News Analysis\n\n"
        report += f"**Overall Sentiment**: {news_sentiment}\n\n"
        report += news_analysis
        report += "\n\n---\n\n"

    # Section 7: Backtesting
    if backtest:
        report += "##  Backtesting Results\n\n"
        report += backtest
        report += "\n---\n\n"

    # Section 8: Reasoning
    if reasoning:
        report += reasoning
        report += "\n---\n\n"

    # Section 9: Validation
    report += "## [OK] Validation & Compliance\n\n"
    report += f"**Status**: {validation_status}\n\n"
    if validation_result:
        # Truncate if too long
        if len(validation_result) > 2000:
            report += validation_result[:2000] + "\n\n*[Truncated for brevity]*\n"
        else:
            report += validation_result

    # Section 10: Warnings (if any)
    if rebalance_warning or rebalance_attempts > 0:
        report += "\n\n---\n\n"
        report += "## [WARN] Notices\n\n"
        if rebalance_attempts > 0:
            report += f"- Portfolio was rebalanced {rebalance_attempts} time(s) due to validation concerns\n"
        if rebalance_warning:
            report += f"- {rebalance_warning}\n"

    # Footer
    report += """
---


  DISCLAIMER: This report is for educational purposes only.
  Consult a qualified financial advisor before investing.

"""

    print("\n" + "="*60)
    print(report)
    print("="*60)

    state['final_report'] = report
    return state

# **Integration**

### Step 1: Update State (TypedDict)

In [35]:
class FinancialPlanState(TypedDict):
    # Original
    user_profile: dict
    macro_data: dict
    screened_assets: dict
    allocation: str
    final_allocation: dict
    top_picks: dict
    macro_regime: str
    reasoning: str
    backtest: str
    backtest_metrics: dict
    validation_result: str
    validation_status: str
    messages: List

    # NEW - Constraint Parser
    parsed_constraints: dict
    excluded_etfs: list
    filtered_universe: dict
    constraint_warnings: list

    # NEW - Research Agent
    research_output: str
    prefer_esg: bool

    # NEW - News Analyzer
    news_analysis: str
    news_sentiment: str

    # NEW - Conditional Loop
    rebalance_attempts: int
    preset_override: str
    rebalance_warning: str
    allocation_method: str

### Step 2: Wire the Workflow

In [36]:
# Create fresh workflow
workflow = StateGraph(FinancialPlanState)

# Add ALL nodes
workflow.add_node("profile", profile_node)
workflow.add_node("constraints", constraint_parser_node)      # NEW
workflow.add_node("macro", macro_node)
workflow.add_node("research", research_agent_node)            # NEW (replaces screener)
workflow.add_node("allocator", enhanced_allocation_node)
workflow.add_node("backtester", backtest_node)
workflow.add_node("news", news_analyzer_node)                 # NEW
workflow.add_node("validator", validation_node)
workflow.add_node("rebalance", rebalance_node)                # NEW
workflow.add_node("reasoning", reasoning_traces_node)
workflow.add_node("reporter", report_node)

# Wire edges
workflow.set_entry_point("profile")
workflow.add_edge("profile", "constraints")
workflow.add_edge("constraints", "macro")
workflow.add_edge("macro", "research")
workflow.add_edge("research", "allocator")
workflow.add_edge("allocator", "backtester")
workflow.add_edge("backtester", "news")
workflow.add_edge("news", "validator")

# Conditional routing after validator
workflow.add_conditional_edges(
    "validator",
    should_continue,
    {
        "rebalance": "rebalance",
        "continue": "reasoning"
    }
)

workflow.add_edge("rebalance", "allocator")  # Loop back
workflow.add_edge("reasoning", "reporter")
workflow.add_edge("reporter", END)

# Compile
app = workflow.compile()

### Step 3: Visualize

In [37]:
# Visualize the graph

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except:
    print("Graph visualization not available. Install pygraphviz or grandalf.")
    print("\nWorkflow structure:")
    print("profile -> constraints -> macro -> research -> allocator -> backtester -> news -> validator")
    print("                                    ^                                         v")
    print("                                     rebalance <- [if concerns] ")
    print("                                                                              v")
    print("                                                              [if OK] -> reasoning -> reporter -> END")

# Better Reporting with PDF + Visualizations

- VISUALIZATION HELPERS

In [38]:
def create_allocation_pie_chart(final_allocation, top_picks):
    """Create pie chart of asset allocation"""
    filtered = {k: v for k, v in final_allocation.items() if v > 0.5}

    labels = [f"{asset}\n({top_picks.get(asset, 'N/A')})"
              for asset in filtered.keys()]
    sizes = list(filtered.values())
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

    fig, ax = plt.subplots(figsize=(10, 7))
    wedges, texts, autotexts = ax.pie(sizes, labels=labels, colors=colors,
                                        autopct='%1.1f%%', startangle=90,
                                        textprops={'fontsize': 11})

    ax.set_title('Portfolio Allocation', fontsize=16, fontweight='bold', pad=20)

    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

    plt.tight_layout()

    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
    buf.seek(0)
    plt.close()

    return buf

def create_backtest_chart(backtest_metrics, portfolio_name="Portfolio"):
    """Create line chart showing portfolio performance vs S&P 500"""

    if not backtest_metrics or '1y' not in backtest_metrics:
        return None

    bt = backtest_metrics['1y']

    days = bt.get('num_days', 252)
    total_return = bt.get('total_return', 0)

    x = np.linspace(0, days, 100)
    portfolio_curve = (1 + total_return/100) ** (x/days) * 100 - 100

    spy_return = 14.18
    spy_curve = (1 + spy_return/100) ** (x/days) * 100 - 100

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(x, portfolio_curve, linewidth=2.5, label=f'{portfolio_name} (+{total_return:.1f}%)',
            color='#4ECDC4')
    ax.plot(x, spy_curve, linewidth=2, label=f'S&P 500 (+{spy_return:.1f}%)',
            color='#FF6B6B', linestyle='--')

    ax.set_xlabel('Days', fontsize=12)
    ax.set_ylabel('Cumulative Return (%)', fontsize=12)
    ax.set_title('1-Year Backtesting Performance', fontsize=16, fontweight='bold')
    ax.legend(fontsize=11, loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

    metrics_text = f"Sharpe: {bt['sharpe_ratio']:.2f}\nMax DD: {bt['max_drawdown']:.1f}%\nVol: {bt['volatility']:.1f}%"
    ax.text(0.98, 0.02, metrics_text, transform=ax.transAxes,
            fontsize=10, verticalalignment='bottom', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.tight_layout()

    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
    buf.seek(0)
    plt.close()

    return buf

def generate_executive_summary(state):
    """Generate LLM executive summary of the plan"""

    profile = state['user_profile']
    final_alloc = state.get('final_allocation', {})
    bt = state.get('backtest_metrics', {}).get('1y', {})
    validation_status = state.get('validation_status', 'Unknown')

    prompt = f"""Write a concise 3-4 sentence executive summary for this financial plan:

Client: {profile['age']}-year-old, {profile['risk_tolerance']} risk tolerance, goal is {profile['goal']} over {profile['time_horizon']} years.

Portfolio: {final_alloc.get('US_Stocks', 0) + final_alloc.get('International_Stocks', 0):.1f}% equities, {final_alloc.get('Bonds', 0):.1f}% bonds, diversified with REITs and commodities.

Performance: 1-year backtest shows {bt.get('total_return', 0):.1f}% return, Sharpe ratio {bt.get('sharpe_ratio', 0):.2f}, max drawdown {bt.get('max_drawdown', 0):.1f}%.

Validation: {validation_status}

Write a professional, confident summary highlighting the key strategy and expected outcomes. Do not use bullet points.
"""

    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

    try:
        response = llm.invoke(prompt)
        return response.content
    except:
        return f"This financial plan recommends a {profile['risk_tolerance']} portfolio for a {profile['age']}-year-old investor pursuing {profile['goal']}."

In [39]:
def generate_pdf_report(state, filename="financial_plan_report.pdf"):
    """Generate comprehensive PDF report with all features"""
    print("[STEP] Generating PDF Report...")

    # Extract state data
    profile = state['user_profile']
    final_alloc = state.get('final_allocation', {})
    top_picks = state.get('top_picks', {})
    macro_regime = state.get('macro_regime', 'Unknown')
    macro_interpretation = state.get('macro_interpretation', '')
    macro_data = state.get('macro_data', {})
    news_analysis = state.get('news_analysis', '')
    news_sentiment = state.get('news_sentiment', 'NEUTRAL')
    backtest_metrics = state.get('backtest_metrics', {})
    validation_result = state.get('validation_result', '')
    validation_status = state.get('validation_status', '')
    parsed_constraints = state.get('parsed_constraints', {})
    excluded_etfs = state.get('excluded_etfs', [])
    research_output = state.get('research_output', '')
    rebalance_attempts = state.get('rebalance_attempts', 0)
    rebalance_warning = state.get('rebalance_warning', '')
    reasoning = state.get('reasoning', '')

    # Create document
    doc = SimpleDocTemplate(filename, pagesize=letter,
                           rightMargin=0.75*inch, leftMargin=0.75*inch,
                           topMargin=0.75*inch, bottomMargin=0.75*inch)

    # Styles
    styles = getSampleStyleSheet()

    title_style = ParagraphStyle(
        'CustomTitle', parent=styles['Heading1'],
        fontSize=24, spaceAfter=30, alignment=TA_CENTER,
        textColor=colors.HexColor('#1a365d')
    )

    section_style = ParagraphStyle(
        'SectionHeader', parent=styles['Heading2'],
        fontSize=14, spaceBefore=20, spaceAfter=10,
        textColor=colors.HexColor('#2c5282')
    )

    subsection_style = ParagraphStyle(
        'SubSection', parent=styles['Heading3'],
        fontSize=12, spaceBefore=15, spaceAfter=8,
        textColor=colors.HexColor('#4a5568')
    )

    body_style = ParagraphStyle(
        'CustomBody', parent=styles['Normal'],
        fontSize=10, spaceAfter=8, alignment=TA_JUSTIFY, leading=14
    )

    highlight_style = ParagraphStyle(
        'Highlight', parent=styles['Normal'],
        fontSize=10, spaceAfter=6,
        backColor=colors.HexColor('#edf2f7'), borderPadding=8
    )

    # Build content
    content = []

    # ===== TITLE PAGE =====
    content.append(Spacer(1, 1*inch))
    content.append(Paragraph("FINANCIAL PLANNING REPORT", title_style))
    content.append(Paragraph("Personalized Portfolio Recommendation", styles['Heading3']))
    content.append(Spacer(1, 0.5*inch))

    # Client info box
    client_info = f"""
    <b>Client Profile</b><br/>
    Age: {profile['age']} years | Risk Tolerance: {profile['risk_tolerance'].title()}<br/>
    Investment Goal: {profile['goal'].title()}<br/>
    Time Horizon: {profile['time_horizon']} years<br/>
    Constraints: {profile.get('constraints', 'None')}
    """
    content.append(Paragraph(client_info, highlight_style))
    content.append(Spacer(1, 0.3*inch))

    # Executive Summary (LLM Generated)
    content.append(Paragraph("Executive Summary", section_style))
    exec_summary = generate_executive_summary(state)
    content.append(Paragraph(exec_summary, body_style))
    content.append(Spacer(1, 0.3*inch))

    # Quick metrics table
    summary_data = [
        ['Market Sentiment', news_sentiment, 'Economic Regime', macro_regime],
        ['Validation Status', '[OK] Approved' if '[OK]' in validation_status else '[WARN] Review', 'Rebalance Attempts', str(rebalance_attempts)]
    ]
    summary_table = Table(summary_data, colWidths=[1.5*inch, 1.3*inch, 1.5*inch, 1.3*inch])
    summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (0, -1), colors.HexColor('#edf2f7')),
        ('BACKGROUND', (2, 0), (2, -1), colors.HexColor('#edf2f7')),
        ('FONTSIZE', (0, 0), (-1, -1), 9),
        ('PADDING', (0, 0), (-1, -1), 6),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.HexColor('#cbd5e0'))
    ]))
    content.append(summary_table)
    content.append(PageBreak())

    # ===== SECTION 1: PORTFOLIO ALLOCATION =====
    content.append(Paragraph("1. Portfolio Allocation", section_style))

    # Allocation table
    alloc_data = [['Asset Class', 'Allocation', 'ETF', 'Fund Name']]
    for asset_class, percentage in sorted(final_alloc.items(), key=lambda x: -x[1]):
        if percentage > 0.5:
            etf = top_picks.get(asset_class, "N/A")
            etf_name = ETF_METADATA.get(etf, {}).get('name', 'N/A')
            alloc_data.append([asset_class.replace('_', ' '), f"{percentage:.1f}%", etf, etf_name])

    alloc_table = Table(alloc_data, colWidths=[1.8*inch, 1*inch, 0.8*inch, 2.4*inch])
    alloc_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2c5282')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('ALIGN', (1, 0), (1, -1), 'CENTER'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('PADDING', (0, 0), (-1, -1), 8),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.HexColor('#cbd5e0')),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f7fafc')])
    ]))
    content.append(alloc_table)
    content.append(Spacer(1, 0.3*inch))

    # Pie chart
    pie_buf = create_allocation_pie_chart(final_alloc, top_picks)
    content.append(Image(pie_buf, width=5*inch, height=3.5*inch))
    content.append(PageBreak())

    # ===== SECTION 2: CONSTRAINT ANALYSIS (NEW) =====
    if parsed_constraints or excluded_etfs:
        content.append(Paragraph("2. Constraint Analysis", section_style))

        constraint_text = ""
        if parsed_constraints.get('exclude_tags'):
            constraint_text += f"<b>Exclusions:</b> {', '.join(parsed_constraints['exclude_tags'])}<br/>"
        if parsed_constraints.get('require_esg'):
            constraint_text += "<b>ESG Preference:</b> Required<br/>"
        if excluded_etfs:
            constraint_text += f"<b>Excluded ETFs ({len(excluded_etfs)}):</b> {', '.join(excluded_etfs)}"

        content.append(Paragraph(constraint_text, body_style))
        content.append(Spacer(1, 0.2*inch))

    # ===== SECTION 3: ECONOMIC ANALYSIS =====
    content.append(Paragraph("3. Economic Analysis", section_style))
    content.append(Paragraph(f"<b>Current Regime:</b> {macro_regime}", body_style))

    if macro_interpretation:
        content.append(Paragraph(f"<b>LLM Analysis:</b> {macro_interpretation}", body_style))

    if macro_data:
        macro_table_data = [['Indicator', 'Value']]
        for indicator, value in macro_data.items():
            macro_table_data.append([indicator.replace('_', ' ').title(), str(value)])

        macro_table = Table(macro_table_data, colWidths=[2*inch, 4*inch])
        macro_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#48bb78')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
            ('FONTSIZE', (0, 0), (-1, -1), 9),
            ('PADDING', (0, 0), (-1, -1), 6),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.HexColor('#cbd5e0'))
        ]))
        content.append(macro_table)
    content.append(Spacer(1, 0.2*inch))

    # ===== SECTION 4: RESEARCH SUMMARY (NEW) =====
    if research_output and "RESEARCH_SUMMARY:" in research_output:
        content.append(Paragraph("4. Research Summary", section_style))
        summary = research_output.split("RESEARCH_SUMMARY:")[1].strip()
        content.append(Paragraph(summary, body_style))
        content.append(Spacer(1, 0.2*inch))

    # ===== SECTION 5: NEWS & SENTIMENT (NEW) =====
    if news_analysis:
        content.append(Paragraph("5. Market News & Sentiment", section_style))
        content.append(Paragraph(f"<b>Overall Sentiment:</b> {news_sentiment}", body_style))

        if "**NEWS SUMMARY:**" in news_analysis:
            start = news_analysis.find("**NEWS SUMMARY:**") + len("**NEWS SUMMARY:**")
            end = news_analysis.find("**SENTIMENT:**")
            if end > start:
                news_summary = news_analysis[start:end].strip()
                content.append(Paragraph(news_summary, body_style))

        if "**KEY RISKS:**" in news_analysis:
            start = news_analysis.find("**KEY RISKS:**") + len("**KEY RISKS:**")
            end = news_analysis.find("**SOURCES:**")
            if end > start:
                risks = news_analysis[start:end].strip().replace('*', '-')
                content.append(Paragraph(f"<b>Key Risks:</b><br/>{risks}", body_style))

    content.append(PageBreak())

    # ===== SECTION 6: BACKTESTING (WITH LINE CHART) =====
    content.append(Paragraph("6. Historical Performance Analysis", section_style))

    # Line chart (Portfolio vs S&P 500)
    backtest_chart = create_backtest_chart(backtest_metrics)
    if backtest_chart:
        content.append(Image(backtest_chart, width=6*inch, height=3*inch))
        content.append(Spacer(1, 0.2*inch))

    # Detailed metrics table
    if backtest_metrics:
        bt_1y = backtest_metrics.get('1y', {})
        bt_3y = backtest_metrics.get('3y', {})

        if bt_1y:
            backtest_data = [
                ['Metric', '1 Year', '3 Years'],
                ['Total Return', f"{bt_1y.get('total_return', 0):.2f}%", f"{bt_3y.get('total_return', 0):.2f}%" if bt_3y else 'N/A'],
                ['Annualized Return', f"{bt_1y.get('annualized_return', 0):.2f}%", f"{bt_3y.get('annualized_return', 0):.2f}%" if bt_3y else 'N/A'],
                ['Sharpe Ratio', f"{bt_1y.get('sharpe_ratio', 0):.2f}", f"{bt_3y.get('sharpe_ratio', 0):.2f}" if bt_3y else 'N/A'],
                ['Max Drawdown', f"{bt_1y.get('max_drawdown', 0):.2f}%", f"{bt_3y.get('max_drawdown', 0):.2f}%" if bt_3y else 'N/A'],
                ['Volatility', f"{bt_1y.get('volatility', 0):.2f}%", 'N/A'],
                ['Best Day', f"{bt_1y.get('best_day', 0):.2f}%", 'N/A'],
                ['Worst Day', f"{bt_1y.get('worst_day', 0):.2f}%", 'N/A']
            ]

            bt_table = Table(backtest_data, colWidths=[2*inch, 1.5*inch, 1.5*inch])
            bt_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2c5282')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
                ('ALIGN', (1, 0), (-1, -1), 'CENTER'),
                ('FONTSIZE', (0, 0), (-1, -1), 10),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('PADDING', (0, 0), (-1, -1), 8),
                ('GRID', (0, 0), (-1, -1), 0.5, colors.HexColor('#cbd5e0')),
                ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.HexColor('#f7fafc')])
            ]))
            content.append(bt_table)

    content.append(PageBreak())

    # ===== SECTION 7: DECISION REASONING =====
    content.append(Paragraph("7. Decision Reasoning", section_style))

    # Profile analysis
    content.append(Paragraph("<b>User Profile Analysis:</b>", subsection_style))
    content.append(Paragraph(f"- <b>Age {profile['age']}</b>: Base stock allocation = {100 - profile['age']}% (100 - age rule)", body_style))

    risk = profile.get('risk_tolerance', 'moderate')
    if risk == 'conservative':
        content.append(Paragraph(f"- <b>Risk Tolerance ({risk})</b>: Reduced equity exposure by 20%", body_style))
    elif risk == 'aggressive':
        content.append(Paragraph(f"- <b>Risk Tolerance ({risk})</b>: Increased equity exposure by 20%", body_style))
    else:
        content.append(Paragraph(f"- <b>Risk Tolerance ({risk})</b>: No adjustment (moderate baseline)", body_style))

    content.append(Paragraph(f"- <b>Goal</b>: {profile['goal']}", body_style))
    content.append(Paragraph(f"- <b>Time Horizon</b>: {profile['time_horizon']} years", body_style))

    # Economic regime reasoning
    content.append(Paragraph("<b>Economic Regime Impact:</b>", subsection_style))
    regime_explanations = {
        "Goldilocks": "Optimal conditions - balanced portfolio approach",
        "Stagflation": "Defensive positioning - increased commodities, reduced equities",
        "Recession": "Flight to safety - increased bonds, reduced risk assets",
        "Strong Growth": "Pro-growth - increased equity allocation",
        "High Inflation": "Inflation hedge - increased commodities and TIPS",
        "Rising Rates": "Rate-sensitive - adjusted bond duration"
    }
    content.append(Paragraph(f"- Regime: {macro_regime} -> {regime_explanations.get(macro_regime, 'Standard allocation')}", body_style))

    # ===== SECTION 8: VALIDATION =====
    content.append(Paragraph("8. Validation & Compliance", section_style))
    status_text = "[OK] APPROVED" if "[OK]" in validation_status else "[WARN] REVIEW RECOMMENDED"
    content.append(Paragraph(f"<b>Status:</b> {status_text}", body_style))

    if validation_result:
        # Extract key validation points
        if "STEP 1" in validation_result:
            content.append(Paragraph("<b>Validation Steps:</b>", subsection_style))
            steps = ["Age Appropriateness", "Goal Alignment", "Risk Tolerance Match", "Constraint Compliance", "Risk Metrics"]
            for i, step in enumerate(steps, 1):
                if f"STEP {i}" in validation_result:
                    verdict = "[OK]" if f"Verdict:** [OK]" in validation_result or f"Verdict:** [OK]" in validation_result else "[WARN]"
                    content.append(Paragraph(f"- Step {i} - {step}: {verdict}", body_style))

    if rebalance_attempts > 0:
        content.append(Paragraph(f"<b>Note:</b> Portfolio was rebalanced {rebalance_attempts} time(s) due to validation concerns.", body_style))

    if rebalance_warning:
        content.append(Paragraph(f"<b>Warning:</b> {rebalance_warning}", body_style))

    # ===== DISCLAIMER =====
    content.append(Spacer(1, 0.5*inch))
    disclaimer_style = ParagraphStyle(
        'Disclaimer', parent=styles['Normal'],
        fontSize=8, textColor=colors.HexColor('#718096'),
        alignment=TA_CENTER, borderWidth=1,
        borderColor=colors.HexColor('#e2e8f0'), borderPadding=10
    )
    disclaimer = """
    <b>DISCLAIMER:</b> This report is generated for educational purposes only and does not constitute
    financial advice. Past performance is not indicative of future results. Consult a qualified
    financial advisor before making investment decisions.
    """
    content.append(Paragraph(disclaimer, disclaimer_style))

    # Build PDF
    doc.build(content)
    print(f"[OK] PDF Report saved: {filename}")
    return filename

In [40]:
# For report
os.makedirs('/mnt/user-data/outputs/', exist_ok=True)

# LangSmith Tracing

In [ ]:
# Set LangSmith credentials
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "<your api key>"
os.environ["LANGCHAIN_PROJECT"] = "FinPlan-LLM"

print("[OK] LangSmith tracing enabled!")
print("View traces at: https://smith.langchain.com/")

[OK] LangSmith tracing enabled!
View traces at: https://smith.langchain.com/


In [42]:
# Check environment variables
print("LANGCHAIN_TRACING_V2:", os.environ.get("LANGCHAIN_TRACING_V2"))
print("LANGCHAIN_API_KEY set:", "Yes" if os.environ.get("LANGCHAIN_API_KEY") else "No")
print("LANGCHAIN_PROJECT:", os.environ.get("LANGCHAIN_PROJECT"))

# Try simple LLM call with tracing

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

print("\nTesting simple LLM call with tracing...")
response = llm.invoke("Say 'tracing works!'")
print(f"Response: {response.content}")

print("\n[OK] If no errors, check LangSmith for this simple call")
print("Go to: https://smith.langchain.com/")

LANGCHAIN_TRACING_V2: true
LANGCHAIN_API_KEY set: Yes
LANGCHAIN_PROJECT: FinPlan-LLM

Testing simple LLM call with tracing...
Response: tracing works!

[OK] If no errors, check LangSmith for this simple call
Go to: https://smith.langchain.com/


# Gradio UI

In [43]:
# Sample profile 1
t1={
  "age": 35,
  "risk_tolerance": "Moderate",
  "goal": "Retirement",
  "horizon": 25,
  "constraints": "ESG focus, no oil companies"
}

In [44]:
# Sample profile 2
t2 = {
  "age": 22,
  "risk_tolerance": "Aggressive",
  "goal": "Growth",
  "horizon": 30,
  "constraints": "No oil, no tobacco, no weapons, no China exposure, ESG only"
}

In [45]:
# Sample profile 3
t3 = {
  "age": 62,
  "risk_tolerance": "Conservative",
  "goal": "Capital Preservation",
  "horizon": 7,
  "constraints": ""
}

In [46]:

def run_financial_planner(age, risk_tolerance, goal, time_horizon, constraints):
    """Run the full pipeline and return output + report"""

    # Add at start of function (after building initial_state)
    tracer = LangChainTracer(project_name="FinPlan-LLM")

    # Build profile
    user_profile = {
        "age": int(age),
        "risk_tolerance": risk_tolerance.lower(),
        "goal": goal,
        "time_horizon": int(time_horizon),
        "constraints": constraints
    }

    initial_state = {
        "user_profile": user_profile,
        "messages": [HumanMessage(content=f"Create a financial plan for: {user_profile}")]
    }

    # Capture print output
    import io
    import sys

    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()

    try:
        # Run workflow
        # final_state = app.invoke(initial_state, config={"recursion_limit": 50})
        # Run workflow
        final_state = app.invoke(initial_state, config={"recursion_limit": 50, "callbacks": [tracer]})

        # Generate PDF
        pdf_path = generate_pdf_report(final_state, "financial_plan_report.pdf")
        # Get LangSmith URL
        run_id = tracer.latest_run.id if tracer.latest_run else None
        if run_id:
            client = Client()
            client.share_run(run_id)
            langsmith_url = f"https://smith.langchain.com/public/{run_id}/r"
        else:
            langsmith_url = None
    finally:
        sys.stdout = old_stdout

    console_output = buffer.getvalue()

    # Build summary output
    summary = f"""
## [OK] Plan Generated Successfully

{"**[ View Full Trace on LangSmith](" + langsmith_url + ")**" if langsmith_url else ""}

**Profile:** {age}yo, {risk_tolerance} risk, {goal}
**Constraints:** {constraints or 'None'}
**Regime:** {final_state.get('macro_regime', 'N/A')}
**Validation:** {final_state.get('validation_status', 'N/A')}

### Portfolio Allocation
{final_state.get('allocation', 'N/A')}

### Console Log
```
{console_output}
```
"""

    return summary, pdf_path

In [47]:
# Build UI
with gr.Blocks(title="FinPlan-LLM", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# [STEP] FinPlan-LLM: AI Financial Advisor")
    gr.Markdown("Enter your details below to generate a personalized financial plan.")

    with gr.Row():
        with gr.Column():
            age = gr.Number(label="Age", value=30, minimum=18, maximum=100)
            risk = gr.Dropdown(
                label="Risk Tolerance",
                choices=["Conservative", "Moderate", "Aggressive"],
                value="Moderate"
            )
            goal = gr.Textbox(label="Investment Goal", value="Retirement planning", placeholder="e.g., Retirement, House down payment")
            horizon = gr.Number(label="Time Horizon (years)", value=20, minimum=1, maximum=50)
            constraints = gr.Textbox(
                label="Constraints (optional)",
                placeholder="e.g., No oil, ESG focus, avoid China exposure",
                value=""
            )
            submit_btn = gr.Button(" Generate Plan", variant="primary")

        with gr.Column():
            output = gr.Markdown(label="Results")
            report_file = gr.File(label="[STEP] Download Report")

    submit_btn.click(
        fn=run_financial_planner,
        inputs=[age, risk, goal, horizon, constraints],
        outputs=[output, report_file]
    )

/tmp/ipython-input-1586396303.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="FinPlan-LLM", theme=gr.themes.Soft()) as demo:


In [48]:
# Launch
demo.launch(share=True)  # share=True gives public URL

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b94a5853748e400827.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
